# Automate importing multiple CSV files into Google BigQuery

# Steps
- import the CSV file into a pandas df
- create a function to clean the table name and remove all extra symbols, spaces, capital letters
- create a function to clean the column headers and remove all extra symbols, spaces, capital letters
- import the data into the db

In [1]:
import os
import numpy as np
import pandas as pd
from google.cloud import bigquery
from pandas.io import gbq

In [21]:
def csv_files():

    #get names of only csv files
    csv_files = []
    for file in os.listdir(os.getcwd()):
        if file.endswith(".csv"):
            csv_files.append(file)

    return csv_files


def configure_dataset_directory(csv_files, dataset_dir):
  
    #make dataset folder to process csv files
    try: 
      mkdir = 'mkdir {0}'.format(dataset_dir)
      os.system(mkdir)
    except:
      pass

    #move csv files to dataset folder
    for csv in csv_files:
      mv_file = "mv '{0}' {1}".format(csv, dataset_dir)
      os.system(mv_file)

    return


def create_df(dataset_dir, csv_files):
  
    data_path = os.getcwd()+'/'+dataset_dir+'/'

    #loop through the files and create the dataframe
    df = {}
    for file in csv_files:
        try:
            df[file] = pd.read_csv(data_path+file)
        except UnicodeDecodeError:
            df[file] = pd.read_csv(data_path+file, encoding="ISO-8859-1") #if utf-8 encoding error
        print(file)
    
    return df


def clean_tbl_name(filename):
  
    #rename csv, force lower case, no spaces, no dashes
    clean_tbl_name = filename.lower().replace(" ", "").replace("-","_").replace(r"/","_").replace("\\","_").replace("$","").replace("%","")
    
    tbl_name = '{0}'.format(clean_tbl_name.split('.')[0])

    return tbl_name



def clean_colname(dataframe):
  
    #force column names to be lower case, no spaces, no dashes
    dataframe.columns = [x.lower().replace(" ", "_").replace("-","_").replace(r"/","_").replace("\\","_").replace(".","_").replace("$","").replace("%","") for x in dataframe.columns]
    
    #processing data
    replacements = {
        'timedelta64[ns]': 'varchar',
        'object': 'varchar',
        'float64': 'float',
        'int64': 'int',
        'datetime64': 'timestamp'
    }

    col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements)))
    
    return col_str, dataframe.columns

def upload_to_db(df,tbl_name):
    df.to_gbq(destination_table= dataset+'.'+ tbl_name,project_id= project_id, if_exists='replace')

In [23]:
# Main

#settings
dataset_dir = 'datasets'
project_id = 'csv-upload-358620' # add project id here
dataset = 'csv_upload' # add dataset name here



#configure environment and create main df
csv_files = csv_files()
configure_dataset_directory(csv_files, dataset_dir)
df = create_df(dataset_dir, csv_files)

for k in csv_files:

    #call dataframe
    dataframe = df[k]

    #clean table name
    tbl_name = clean_tbl_name(k)
    
    #clean column names
    col_str, dataframe.columns = clean_colname(dataframe)
    
    #upload to GBQ
    upload_to_db(dataframe,tbl_name)

Customer Contracts$.csv
Customer Engagements.csv
Customer Demo.csv


100%|██████████| 1/1 [00:00<00:00, 2118.34it/s]
